#### Concepts

#### Name Mangling

In [6]:
class Person:
    __number = 0
    def __init__(self, name):
        self.name = name
        self.age = Person.__number # self.__number
    def introduce(self):
        print('The person is '+str(self.name)+' and is '+str(self.age))
class Student(Person):
    __number = 10
    school = 'Omega'
    def __init__(self, name, age, grade):
        # Person.__init__(self)
        super(Student, self).__init__(name, age)
        self.grade = grade
    def introduce(self):
        print('The student is '+str(self.name)+' and is '+str(self.age)+' in grade '+str(self.grade))
print(Student._Student__number)
print(Student._Person__number)
person = Person('jeff')
print(person._Person__number)

10
0
0


Name mangling convert \__number to \_Person\__number which belongs to class Person, but class Student will also inherit it, however there won't be same-name-variable-overwrite issue.

#### Inheritance

In [11]:
class Base(object):
    def __init__(self):
        print("enter Base")
        print("leave Base")
    def test(self):
        print("test Base")
 
class A(Base):
    def __init__(self):
        print("enter A")
        super(A, self).__init__()
        print("leave A")
 
class B(Base):
    def __init__(self):
        print("enter B")
        super(B, self).__init__()
        print("leave B")
    def test(self):
        print("test B")
 
class C(A, B):
    def __init__(self):
        print("enter C")
        print(C.__mro__)
        super(C, self).__init__()
        print("leave C")
        
c = C()  # 只要父类们都分别正常执行super().__init__()，super会按照__mro__的顺序执行所有的初始化方法
c.test()  # 会按照__mro__的顺序，找到第一个test方法并执行

enter C
(<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>)
enter A
enter B
enter Base
leave Base
leave B
leave A
leave C
test B


\__new\__ vs \__init\__

\__new\__ is a static method, while \__init\__ is a instance method.
\__new\__ returns an instance of class, \__init\__ does not.
\__init\__ is invoked after \__new\__ returns an instance of cls.
\__new\__ is for creating an instance whereas \__init\__ is for initiating the instance.

In [ ]:
class Mapping:
    def __init__(self, iterable):
        self.items_list = []
        self.__update(iterable)

    def update(self, iterable):
        for item in iterable:
            self.items_list.append(item)

    __update = update   # private copy of original update() method

class MappingSubclass(Mapping):

    def update(self, keys, values):
        # provides new signature for update()
        # but does not break __init__()
        for item in zip(keys, values):
            self.items_list.append(item)

So the difference between vars(x) and dir(x) is that dir(x) does the extra work of looking in x's class (and its bases) for attributes that are accessible from it, not just those attributes that are stored in x's own symbol table.

In [8]:
p = Person('tom')
print(dir(Student))
print(vars(Student))

['_Person__number', '_Student__number', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'introduce', 'school']
{'__module__': '__main__', '__doc__': None, '_Student__number': 10, 'school': 'Omega', 'introduce': <function Student.introduce at 0x105a56048>, '__init__': <function Student.__init__ at 0x105a56158>}


\__slots__

\__slots\__ is used to limit attributes of instance of a class although its subclass is not limited to parent class's \__slots\__ attribute. Once the subclass has \__slots\__ too, it's limited to both its parent class's \__slots\__ plus its own.
And instance and class attribute are different, so cannot delete class attribute through its instance even the instance can have attribute of its class.

In [14]:
class Person:
    number = 0
    __slots__ = ['name', 'age']
    def __init__(self, name, age):
        self.name = name
        self.age = age

p = Person('linoel', 12)
p.sex = 'male' # cannot add because of __slots__
del p.number # instance cannot delete class's attribute

AttributeError: 'Person' object attribute 'number' is read-only

Descriptor

In [6]:
class demo:
    def __init__(self, x):
        self.foo = x
        
    def foo(self):
        print('function')
    
d = demo(10)
print(d.__dict__)
d.foo # instance attribute priors non-data descriptor

{'foo': 10}


10

In [36]:
class MyDescriptor:
    def __init__(self, name):
        self.name = name
    def __get__(self, obj, objtype):
        print('getting from descriptor...')
        return self.name
    def __set__(self, obj, value):
        print('setting via descriptor...')
        self.name = value
class Person:
    number = MyDescriptor(0)
    def __init__(self, name, age, number):
        self.name = name
        self.age = age
        self.number = number # attribute assigning via MyDescriptor
        # self.__dict__['number'] = number # result is 0 instead of 1, prove that data descriptor prior to instance dict
        print('init')
    def __len__(self):
        return 10
    def __getattribute__(self, name): # will only be called for instance of class, not the class
        print('invoke __getattribute__')
        return super(Person, self).__getattribute__(name)
        #return object.__getattribute__(self, name) another way to do it
    def __introduce(self):
        print('The person is '+str(self.name)+' and is '+str(self.age))
p = Person('m', 11, 1)
p.number # attribute access will start from instance's dict, then its class or parent class's dict
# Person.number == Person.__dict__['number'].__get__(None, Person) # True, both will call MyDescriptor, but not __getattribute__
#print(Person.__len__) # will not call __getattribute__, will call __getattribute__ in its metaclass
#print(p.__len__) # will call __getattribute__

setting via descriptor...
init
invoke __getattribute__
getting from descriptor...


1

延迟计算

In [35]:
import profile

def fab(n):
    return 1 if n in [0, 1] else fab(n-1) + fab(n-2)

profile.run('fab(20)')

class cache:
    def __init__(self, func):
        self.func = func
        self.results = {}
        
    def __call__(self, n, *args):
        try:
            return self.results[n]
        except:
            self.results[n] = self.func(n, *args)
            return self.results[n]
        
@cache
def fab(n):
    return 1 if n in [0, 1] else fab(n-1) + fab(n-2)

profile.run('fab(20)')

         21895 function calls (5 primitive calls) in 0.033 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.033    0.033 :0(exec)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
  21891/1    0.033    0.000    0.033    0.033 <ipython-input-35-759f9c3bba86>:3(fab)
        1    0.000    0.000    0.033    0.033 <string>:1(<module>)
        1    0.000    0.000    0.033    0.033 profile:0(fab(20))
        0    0.000             0.000          profile:0(profiler)


         64 function calls (6 primitive calls) in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 :0(exec)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
     39/1    0.000    0.000    0.000    0.000 <ipython-input-35-759f9c3bba86>:13(__call__)
     21/1    0.000    0.000    0.000    0.000 <ipython-

#### metaclass

type can be used to create class.
In fact, class keyword invokes type to create class.

In [33]:
def func(self, str='world'):
    print('hello %s' % str)

Hello = type('Hello', (object,), dict(hello=func))
h = Hello()
h.hello()

hello world


use metaclass to define class creation.
class can be seen as instance of metaclass.

In [34]:
class ListMetaclass(type):
    def __new__(cls, name, bases, attrs): # here cls refers to ListMetaclass
        attrs['add'] = lambda self, value: self.append(value)
        return type.__new__(cls, name, bases, attrs)

class MyList(list, metaclass=ListMetaclass):
    pass

l = MyList()
l.add(1)
print(l)

[1]


#### type vs object

object is as the base class of all classes, including type.
type defines the type of all classes, including itself.

In [35]:
print(object.__bases__)
print(object.__class__)
print(type.__bases__)
print(type.__class__)

()
<class 'type'>
(<class 'object'>,)
<class 'type'>
